Christian Sutton
Module 9
10/31/20   

# Problem 1
**[10 pts] Describe the environment in the Nim learning model.**  

The environment consists the rules of the deterministic NIM game. Only pick from 1 pile. Must pick at least 1 item. Last player to pick up the last item wins

# Problem 2
**[10 pts] Describe the agent in the Nim learning model.)**  
The agent in the NIM game is the Qlearner. The Qlearner tablulates all possible game states and learns the best next move through rewards and penalties in the Qtable during each game state win/ or potential loss 

# Problem 3  
**[10 pts] Describe the reward and penalty in the Nim learning model.**  
The reward is created by updating the table with a reward value in the event that a game is won. 
a penalty is created when the table is decremented with a penalty value when the current move will lead to a loss in the next game state.


# Problem 4  
**[10 pts] How many possible states there could be in the Nim game with a maximum of 10
items per pile and 3 piles total?**  
There are 11^3 states

# Problem 5  
**[10 pts] How many possible actions there could be in the Nim game with 10 items per pile
and 3 piles total?**  

Assuming the smallest possible move size, there are 30 possible actions. 

# Problem 6  
**[50 pts] Find a way to improve the provided Nim game learning model. Do you think one
can beat the Guru player? (Hint: How about penalizing the losses? Hint: It is indeed
possible to find a better solution which improves the way Q-learning updates its Q-table).**  

My solution is below, there is some kind bug that isn't allowing the penalties to have an effect on Qlearner winning more versus Guru player

In [1]:
import numpy as np
from random import randint, choice

# The number of piles is 3


# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game():
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(st):
    xored = st[0] ^ st[1] ^ st[2]
    if xored == 0:
        return nim_random(st)
    #
    for pile in range(3):
        s = st[pile] ^ xored
        if s <= st[pile]:
            return st[pile]-s, pile

# Random Nim player
def nim_random(_st):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

In [2]:
def nim_qlearner(_st):
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    #
    return move, pile  # action

In [3]:
Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(a, b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[a] if side == 'A' else Engines[b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side
        #
        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n, a, b):
    from collections import defaultdict
    wins = defaultdict(int)
    for i in range(_n):
        wins[game(a, b)] += 1
    # info
    print(f"{_n} games, {a:>8s} {wins['A']}   {b:>8s} {wins['B']}")
    #
    return wins['A'], wins['B']

In [4]:
qtable, Alpha, Gamma, Reward = None, 1.0, 0.8, 100.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break  # new game
            elif np.max(qtable[st2[0], st2[1], st2[2]])== Reward:
                qtable_update(-Reward, st1, move, pile, np.min(qtable[st2[0], st2[1], st2[2]]))
            else: 
                qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            st1 = st2

# Equation 3 - update the qtable
def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

In [5]:
%%time
nim_qlearn(100000)

Wall time: 15.8 s


In [6]:
%%time
# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
wins = []
for n in n_train:
    #nim_qlearn(n)
    a, b = play_games(n, 'Qlearner', 'Guru')
    wins += [a/(a+b)]

3 games, Qlearner 0       Guru 3
10 games, Qlearner 0       Guru 10
100 games, Qlearner 1       Guru 99
1000 games, Qlearner 3       Guru 997
10000 games, Qlearner 50       Guru 9950
50000 games, Qlearner 271       Guru 49729
100000 games, Qlearner 534       Guru 99466
Wall time: 11.4 s


In [7]:
# Function to print the entire set of states
def qtable_log(_fn):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)
        #
        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r
            #
            print(s, file=fout)
#
qtable_log('C:/Users/physi/Desktop/AppliedMachineLearning_EN.705.601/Mod9/qtable_debug.txt4')